
# **SMISHING DETECTION**

---

---





In [233]:
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import roc_curve
#from sklearn.metrics import balanced_accuracy_score
#from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [234]:
DATA_PATH = "data/smsdb.txt"
COLUMN_LABEL = "label"
COLUMN_TEXT = "text"
LABEL_LEGIT = 'LEGI'
LABEL_SPAM = 'SPAM'
LABEL_SMISHING = 'SMIS'

dataset = pd.read_csv(DATA_PATH, sep='\t', names=[COLUMN_LABEL, COLUMN_TEXT], header=None)
print('Total size:', dataset.shape[0])
print('Legit messages:', dataset[dataset[COLUMN_LABEL] == LABEL_LEGIT].shape[0])
print('Spam messages:', dataset[dataset[COLUMN_LABEL] == LABEL_SPAM].shape[0])
print('Smishing messages:', dataset[dataset[COLUMN_LABEL] == LABEL_SMISHING].shape[0])

Total size: 1533
Legit messages: 1055
Spam messages: 176
Smishing messages: 302


In [235]:
dataset = dataset[((dataset[COLUMN_LABEL] == LABEL_LEGIT) | (dataset[COLUMN_LABEL] == LABEL_SMISHING))]
print('Spam messages:', dataset[dataset[COLUMN_LABEL] == LABEL_SPAM].shape[0])
print(len(dataset))

Spam messages: 0
1357


In [236]:
def messages2vectors(messages):
    '''
    Transforms single message into feature-vector;
    Parameters:
        messages    -   array of strings;
    Returns:
        features    -   array of feature-vectors;   
    '''

    elmo = hub.Module("https://tfhub.dev/google/elmo/1")

    features = np.zeros((0, 1024))
    n = 100
    l = int(len(messages) / n) if len(messages) % n == 0 else int(len(messages) / n) + 1
    for i in range(l):

        if (i + 1) * n < len(messages):
            right = (i + 1) * n
            embedds = elmo(messages[int(i * n) : right], signature="default", as_dict=True)["default"] 
        else:
            embedds = elmo(messages[:len(messages) - int(i * n)], signature="default", as_dict=True)["default"] 

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            embedds = sess.run(embedds)
            features = np.concatenate([features, embedds])

    return features

def convert_labels(labels_raw):
    labels = []

    for i in labels_raw:
      if i == "LEGI":
        labels.append(0)
      elif i == "SMIS":
        labels.append(1)
      else:
        pass
    labels = np.asarray(labels)

    return labels

In [237]:
features = messages2vectors(dataset[COLUMN_TEXT])
labels = convert_labels(dataset[COLUMN_LABEL])
#print(features.shape)
#print(labels.shape)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [238]:
def get_metrics(labels, predictions):
    '''
    FAR -   False Acceptance Rate;
    FRR -   False Rejection Rate;
    FPR = FP/(FP + TN)
    FNR = FN/(FN + TP)
    where FP: False positive
          FN: False Negative
          TN: True Negative
          TP: True Positive  
    '''  
    
    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()
    #print("tn, fp, fn, tp", tn, fp, fn, tp)

    # >FAR like FNR
    FRR = fp / (fp + tn) * 100
    FAR = fn / (fn + tp) * 100
    return FAR, FRR

In [239]:
train_data, test_data, train_labels, test_labels = train_test_split(features, labels, test_size=0.7, random_state=5)  # 70% training and 30% test

**Random Forests Classifiers**

In [240]:
clf=RandomForestClassifier()
clf.fit(train_data, train_labels)
test_labels_pred = clf.predict(test_data)
train_labels_pred = clf.predict(train_data)

In [241]:
print("Accuracy test", accuracy_score(test_labels, test_labels_pred))
print("Accuracy train", accuracy_score(train_labels, train_labels_pred))
trainFAR, trainFRR = get_metrics(train_labels, train_labels_pred)
testFAR, testFRR = get_metrics(test_labels, test_labels_pred)

Accuracy test 0.9621052631578947
Accuracy train 1.0


In [242]:
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

Train:
	FAR: 0.0
	FRR: 0.0
Test:
	FAR: 10.83743842364532
	FRR: 1.8741633199464525


 **SVM**

In [243]:
clf = svm.SVC(kernel='linear')
clf.fit(train_data, train_labels)
test_labels_pred = clf.predict(test_data)
train_labels_pred = clf.predict(train_data)

In [244]:
print("Accuracy test", accuracy_score(test_labels, test_labels_pred))
print("Accuracy train", accuracy_score(train_labels, train_labels_pred))

Accuracy test 0.9747368421052631
Accuracy train 1.0


In [245]:
trainFAR, trainFRR = get_metrics(train_labels, train_labels_pred)
testFAR, testFRR = get_metrics(test_labels, test_labels_pred)

In [246]:
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

Train:
	FAR: 0.0
	FRR: 0.0
Test:
	FAR: 3.4482758620689653
	FRR: 2.2757697456492636


**K-Nearest Neighbors**

In [247]:
from sklearn.neighbors import KNeighborsClassifier

In [248]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_data, train_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [249]:
test_labels_pred = knn.predict(test_data)
train_labels_pred = knn.predict(train_data)

In [250]:
print("Accuracy test", accuracy_score(test_labels, test_labels_pred))
print("Accuracy train", accuracy_score(train_labels, train_labels_pred))

Accuracy test 0.9673684210526315
Accuracy train 0.9754299754299754


In [251]:
trainFAR, trainFRR = get_metrics(train_labels, train_labels_pred)
testFAR, testFRR = get_metrics(test_labels, test_labels_pred)
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

Train:
	FAR: 3.0303030303030303
	FRR: 2.272727272727273
Test:
	FAR: 1.477832512315271
	FRR: 3.748326639892905


**Logistic Regression**

In [252]:
lg = LogisticRegression()

In [253]:
lg.fit(train_data, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [254]:
test_labels_pred = lg.predict(test_data)
train_labels_pred = lg.predict(train_data)

In [255]:
print("Accuracy test", accuracy_score(test_labels, test_labels_pred))
print("Accuracy train", accuracy_score(train_labels, train_labels_pred))

Accuracy test 0.9778947368421053
Accuracy train 1.0


In [256]:
trainFAR, trainFRR = get_metrics(train_labels, train_labels_pred)
testFAR, testFRR = get_metrics(test_labels, test_labels_pred)
print('Train:')
print('\tFAR:', trainFAR)
print('\tFRR:', trainFRR)

print('Test:')
print('\tFAR:', testFAR)
print('\tFRR:', testFRR)

Train:
	FAR: 0.0
	FRR: 0.0
Test:
	FAR: 2.955665024630542
	FRR: 2.0080321285140563
